<h1>#####      insertOne() avec MongoDB & Python (pymongo)              #####</h1>

In [61]:
import pymongo
import time
import json

In [62]:
start = time.time()

try:
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    client.server_info()
except:
    print('Authentification failed ')

print(time.time() - start)

0.0239713191986084


<h3>Créer 1ère DB "metflix" qui contient 1collection "movies" dans laquelle vs inserez les documents suivants :</h3>
<ul>
<li>title = Rocky ; year = 1976</li>
<li>title = Rocky ; year = 1976 ; _id = tt0075148</li>
<li>title = Rocky ; year = 1976 ; _id = tt0075148</li>
<li>title = Rocky ; year = 1976</li>
</ul>

In [63]:
db = client.metflix
db.movies.drop()
movies = db.movies
print(db.list_collection_names())

['users']


In [64]:
try :   
    movies.insert_one({"title": "Rocky", "year": 1976})
    # Erreur car l'id doit être unique 
    # movies.insert_one({"title": "Rocky", "year": 1976, "_id": "tt0075148"})
    # movies.insert_one({"title": "Rocky", "year": 1976, "_id": "tt0075148"})
    movies.insert_one({"title": "Rocky", "year": 1976})
except :
    print("Insert failed")

items = movies.find()
for item in items :
    print(item)

{'_id': ObjectId('600477dd95dd1b64a4d1a7c1'), 'title': 'Rocky', 'year': 1976}
{'_id': ObjectId('600477dd95dd1b64a4d1a7c2'), 'title': 'Rocky', 'year': 1976}


<h3>Et une autre collection "users"</h3>
<p>Qui contient un document qui porte votre prénom et NOM</p>

In [65]:
users = db.users

try : 
    users.insert_one({"firstname": "Jérémy", "lastname": "RICHARD"})
except : 
    print("User insertion failed")

<h1>#####                         insertMany()                   #####</h1>

<h2>Supp la dernière collection, la recréer en y insérant les documents suivants :</h2>
<ul>
    <li>title = Rocky, year = 1976, _id = tt0075148</li>
    <li>title = Jaws, year = 1975, imdb = tt0073195</li>
    <li>title = Mad Max 2 : The Road Warrior, year = 1981, imdb = tt0082694</li>
    <li>title = Raiders of the Lost Ark, year = 1981, imdb = tt0082971</li>
</ul>

In [66]:
list = [{
        "title": "Rocky", 
        "year": 1976,
        "_id": "tt0075148"
    },
    {
        "title": "Jaws", 
        "year": 1975,
        "imdb": "tt0073195"
    },
    {
        "title": "Mad Max 2 : The road Warrior", 
        "year": 1981,
        "imdb": "tt0082694"
    },
    {
        "title": "Test", 
        "year": 1981,
        "imdb": "tt0082971"
    }
 ]

try : 
    movies.insert_many(list)
except :
    print("Insert many failed")

<h2>Afficher ts les documents de la collection movies, le tout 1è document, le(s) document(s) où on trouve les films produits en 1981, le document où on trouve le film qui porte le titre "Jaws" ?</h2>

In [67]:
movies.find_one()

query = {'year': 1981}
query = {"title": "Jaws"}

items = movies.find(query)
for item in items :
    print(item)

{'_id': ObjectId('600477de95dd1b64a4d1a7c4'), 'title': 'Jaws', 'year': 1975, 'imdb': 'tt0073195'}


<h1>#####               Import de fichiers .json ?                      #####</h1>

<h2>Créer une DB movies_artists avec deux collections : une pour les films et l’autre pour les artistes (voir les fichiers artists.json et movies.json associés) ?</h2>

In [68]:
db = client.movies_artists

try :
    with open('json/artists.json') as json_data:
        data_artists = json.load(json_data)

    with open('json/movies.json') as json_data:
        data_movies = json.load(json_data)
except :
    print("impossible d'ouvrir les fichiers json")

<h3>Supprimer la collection si elle existe (pour qu'elle soit recréer de nouveau et y injecter de la data fraîche).</h3>

In [69]:
if db.artist.find() : 
    db.artists.drop()

if db.movies.find():
    db.movies.drop()

movies = db.movies
artists = db.artists

try:
    db.artists.insert_many(data_artists)
    db.movies.insert_many(data_movies)
except:
    print("Insert many error")

<h2>Augmenter la collection movies avec le fichier movies_suite.json</h2>

In [70]:
try :
    with open('json/artists_suite.json') as json_data:
        data_artists = json.load(json_data)

    with open('json/movies_suite.json') as json_data:
        data_movies = json.load(json_data)
except :
    print("impossible d'ouvrir les fichiers json")

try:
    db.artists.insert_many(data_artists)
    db.movies.insert_many(data_movies)
except:
    print("Insert many error")

<h1>#####               Pagination des résultats ?                      #####</h1>

<h2>Afficher les 12 documents (films) à partir du dixième inclus en se limitant aux title et _id ?</h2>

In [71]:
movies = db.movies.find({},{"title" : 1}).limit(12).skip(9)
for movie in movies:
    print(movie)

{'_id': 'movie:10', 'title': 'Blade Runner'}
{'_id': 'movie:11', 'title': 'PiÃ¨ge de cristal'}
{'_id': 'movie:12', 'title': '58 minutes pour vivre'}
{'_id': 'movie:13', 'title': 'Van Gogh'}
{'_id': 'movie:14', 'title': 'Seven'}
{'_id': 'movie:15', 'title': 'Twelve Monkeys'}
{'_id': 'movie:16', 'title': 'Le last_name de la rose'}
{'_id': 'movie:17', 'title': 'Pulp fiction'}
{'_id': 'movie:18', 'title': 'Mary Ã\xa0 tout prix'}
{'_id': 'movie:19', 'title': 'Terminator'}
{'_id': 'movie:20', 'title': 'Les dents de la mer'}
{'_id': 'movie:21', 'title': 'Le silence des agneaux'}


<h1>#####                  Trier les resultats ?                        #####</h1>

<h2>Trier les documents (de la dernière qry) sur le titre du film ds un ordre croissant (ensuite décroissant) ?</h2>

In [72]:
ascendingTitle = db.movies.find({},{"title" : 1}).sort("title", 1)
descendingTitle = db.movies.find({},{"title" : 1}).sort("title", -1)

print("Ordre Croissant")
for movie in ascendingTitle:
    print(movie)

print("Ordre Décroissant")
for movie in descendingTitle:
    print(movie)



Ordre Croissant
{'_id': 'movie:12', 'title': '58 minutes pour vivre'}
{'_id': 'movie:2', 'title': 'Alien'}
{'_id': 'movie:7', 'title': 'American Beauty'}
{'_id': 'movie:53', 'title': 'Bad Lieutenant'}
{'_id': 'movie:88', 'title': 'Batman begins'}
{'_id': 'movie:10', 'title': 'Blade Runner'}
{'_id': 'movie:72', 'title': 'Casino'}
{'_id': 'movie:73', 'title': 'Casino Royale'}
{'_id': 'movie:51', 'title': 'De bruit et de fureur'}
{'_id': 'movie:78', 'title': 'Django unchained'}
{'_id': 'movie:37', 'title': 'Eyes Wide Shut'}
{'_id': 'movie:77', 'title': 'Fargo'}
{'_id': 'movie:40', 'title': 'FenÃªtre sur cour'}
{'_id': 'movie:9', 'title': 'Gladiator'}
{'_id': 'movie:22', 'title': 'Godzilla'}
{'_id': 'movie:69', 'title': 'Heat'}
{'_id': 'movie:8', 'title': 'Impitoyable'}
{'_id': 'movie:86', 'title': 'Inception'}
{'_id': 'movie:79', 'title': 'Inglourious Basterds'}
{'_id': 'movie:80', 'title': 'Inglourious Basterds'}
{'_id': 'movie:85', 'title': 'Interstellar'}
{'_id': 'movie:57', 'title': '

<h1>#####             Critères de recherche simples ?         #####</h1>

<h2>Afficher les infos du film dont l'id est 2 ? Se limiter aux infos relatives au titre, au genre et au summary ?</h2>

In [73]:
movie1 = db.movies.find_one({"_id": "movie:2"},{"title" : 1, "genre": 1, "summary": 1 })
print(movie1)

{'_id': 'movie:2', 'title': 'Alien', 'genre': 'Science-fiction', 'summary': 'PrÃ¨s d\'un vaisseau spatial Ã©chouÃ© sur une lointaine planÃ¨te, des Terriens en mission dÃ©couvrent de bien Ã©tranges "oeufs". Ils en ramÃ¨nent un Ã\xa0 bord, ignorant qu\'ils viennent d\'introduire parmi eux un huitiÃ¨me passager particuliÃ¨rement fÃ©roce et meurtrier. '}


<h2>Combien de films ont été produit en en 1979 et les afficher en se limitant aux infos relatives au titre, à l'année et à la liste des acteurs suivant un ordre alphabétique décroissant sur le titre.</h2>

In [74]:
allMoviesOf1979 = db.movies.find({"year": 1979},{"title" : 1, "year": 1, "actors": 1 }).sort("title", -1)
for movie in allMoviesOf1979 :
    print(movie)

{'_id': 'movie:31', 'title': 'Les bronzÃ©s font du ski', 'year': 1979, 'actors': [{'_id': 'artist:75', 'role': 'Nathalie Morin'}, {'_id': 'artist:76', 'role': 'Christiane'}, {'_id': 'artist:84', 'role': 'Jean-Claude Dus'}, {'_id': 'artist:85', 'role': 'JÃ©rÃ´me'}, {'_id': 'artist:86', 'role': 'Popeye'}]}
{'_id': 'movie:2', 'title': 'Alien', 'year': 1979, 'actors': [{'_id': 'artist:5', 'role': 'Ripley'}]}


<h2>Afficher les infos sur le film dont le genre est "Science-fiction" et produit en 1979 sans préciser l'_id et le summary ?</h2>

In [75]:
scienceMovie = db.movies.find({"year": 1979, "genre" : "Science-fiction"},{"_id" : 0, "summary": 0})
for movie in scienceMovie:
    print(movie)

{'title': 'Alien', 'year': 1979, 'genre': 'Science-fiction', 'country': 'USA', 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:5', 'role': 'Ripley'}]}


<h1>#####             Critères de recherche complexes ?                 #####</h1>

<h2>Combien de films produits par le director dont l'id est 4 et les afficher de plus récent au moins récent (sans le résumé ni le country ni le genre) ?</h2>

In [76]:
director4Movies = db.movies.find({"director":{"_id" : "artist:4"}},{"genre" : 0, "summary": 0, "country": 0}).sort("year", 1)
for movie in director4Movies:
    print(movie)

{'_id': 'movie:2', 'title': 'Alien', 'year': 1979, 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:5', 'role': 'Ripley'}]}
{'_id': 'movie:10', 'title': 'Blade Runner', 'year': 1982, 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:24', 'role': 'Deckard'}, {'_id': 'artist:25', 'role': 'Batty'}]}
{'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000, 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:23', 'role': 'Maximus'}, {'_id': 'artist:147', 'role': 'Commode'}, {'_id': 'artist:148', 'role': 'Lucilla'}, {'_id': 'artist:149', 'role': 'Marc Aurele'}]}


<h2>Combien de films produits par le director qui porte l'id 4 et dont l'un des acteurs a joué le rôle de Maximus et les afficher sans le résumé ni le country ni le genre ?</h2>

In [77]:
movieByDirector4AndMaximus = db.movies.find({"director":{"_id" : "artist:4"}, "actors.role": "Maximus"},{"genre" : 0, "summary": 0, "country": 0}).sort("year", 1)
for movie in movieByDirector4AndMaximus:
    print(movie)

{'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000, 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:23', 'role': 'Maximus'}, {'_id': 'artist:147', 'role': 'Commode'}, {'_id': 'artist:148', 'role': 'Lucilla'}, {'_id': 'artist:149', 'role': 'Marc Aurele'}]}


<h1>#####           Critère de recherche par intervalles                #####</h1>

<h2>Ts les films produits après 2000 compris et avant 2005 exlus (dans un ordre chronologique) en se limitant au titre du film et son année de production ?</h2>

In [78]:
moviesBetween2000and2005 = db.movies.find({"year":{"$gte":2000,"$lt":2005}},{"title" : 1, "year": 1}).sort("year", 1)
for movie in moviesBetween2000and2005:
    print(movie)

{'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000}
{'_id': 'movie:87', 'title': 'Memento', 'year': 2000}
{'_id': 'movie:62', 'title': 'Stalingrad', 'year': 2001}
{'_id': 'movie:47', 'title': 'Spider-Man', 'year': 2002}
{'_id': 'movie:49', 'title': 'The Matrix reloaded', 'year': 2003}
{'_id': 'movie:50', 'title': 'The Matrix Revolutions', 'year': 2003}
{'_id': 'movie:60', 'title': 'Lost in Translation', 'year': 2003}
{'_id': 'movie:61', 'title': 'Kill Bill', 'year': 2003}


<h1>#####               Opérateurs booléens                             #####</h1>

<h2>Lister les films parus en 1997 ou avec l'acteur id = 147 ?</h2>

In [79]:
moviesWithActor147In1997 = db.movies.find({"$or" : [{"year": 1997}, {"actors._id": "artist:147"}] },{"title" : 1, "year": 1, "actors":1})

for movie in moviesWithActor147In1997:
    print(movie)

{'_id': 'movie:3', 'title': 'Titanic', 'year': 1997, 'actors': [{'_id': 'artist:109', 'role': 'Rose DeWitt Bukater'}, {'_id': 'artist:110', 'role': 'Jack Dawson'}]}
{'_id': 'movie:5', 'title': 'Volte/Face', 'year': 1997, 'actors': [{'_id': 'artist:11', 'role': 'Sean Archer/Castor Troy'}, {'_id': 'artist:12', 'role': 'Castor Troy/Sean Archer'}]}
{'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000, 'actors': [{'_id': 'artist:23', 'role': 'Maximus'}, {'_id': 'artist:147', 'role': 'Commode'}, {'_id': 'artist:148', 'role': 'Lucilla'}, {'_id': 'artist:149', 'role': 'Marc Aurele'}]}
{'_id': 'movie:26', 'title': 'Les pleins pouvoirs', 'year': 1997, 'actors': [{'_id': 'artist:20', 'role': 'Luther Whitney'}, {'_id': 'artist:21', 'role': 'Le prÃ©sident Richmond'}, {'_id': 'artist:69', 'role': 'Seth Frank'}]}
{'_id': 'movie:28', 'title': 'Le monde perdu', 'year': 1997, 'actors': [{'_id': 'artist:57', 'role': 'Dr. Ian Malcolm'}]}
{'_id': 'movie:43', 'title': 'Le cinquiÃ¨me Ã©lÃ©ment', 'year': 199

<h1>#####       Opérateurs ensemblistes : in, not in, any, all, exists  #####</h1>

<h2>Q : Combien de films dans lesquels ont joué au moins un des artistes suivants : 34, 98 et 1 et afficher leur titre et la liste des actors ; sens du tri : chronologique et alphabétique croissant sur les noms des artistes ?</h2>

In [80]:
moviesWithSpecifiedActors = db.movies.find({"actors._id":{"$in":["artist:34","artist:98","artist:1"]}},{"title" : 1, "actors": 1}).sort("year", 1).sort("actors",1)

for movie in moviesWithSpecifiedActors:
    print(movie)
    print("\n")

{'_id': 'movie:56', 'title': 'Le parrain III', 'actors': [{'_id': 'artist:1', 'role': 'Mary Corleone'}, {'_id': 'artist:176', 'role': 'Michael Corleone'}, {'_id': 'artist:183', 'role': 'Kay Adams'}, {'_id': 'artist:186', 'role': 'Vincent Mancini'}]}


{'_id': 'movie:36', 'title': 'Reservoir dogs', 'actors': [{'_id': 'artist:37', 'role': 'Mr. Brown'}, {'_id': 'artist:98', 'role': 'Mr. White/Larry'}, {'_id': 'artist:99', 'role': 'Freddy Newendyke/Mr. Orange'}, {'_id': 'artist:100', 'role': 'Nice Guy Eddie'}, {'_id': 'artist:215', 'role': 'Mr Blonde'}, {'_id': 'artist:217', 'role': 'Mr Pink'}, {'_id': 'artist:218', 'role': 'Mr Blue'}]}


{'_id': 'movie:17', 'title': 'Pulp fiction', 'actors': [{'_id': 'artist:11', 'role': 'Vincent Vega'}, {'_id': 'artist:27', 'role': 'Butch Coolidge'}, {'_id': 'artist:37', 'role': 'Jimmy Dimmick'}, {'_id': 'artist:38', 'role': 'Jules Winnfield'}, {'_id': 'artist:39', 'role': 'Jody'}, {'_id': 'artist:40', 'role': 'Mia Wallace'}, {'_id': 'artist:97', 'role':

<h2>Q : Chercher les films dans lesquels ont joué ensemble les actors 23 et 147 ?</h2>


In [81]:
moviesWithSpecifiedActors2 = db.movies.find({"actors._id":{"$all":["artist:23","artist:147"]}},{"title" : 1, "actors": 1}).sort("title",1)

for movie in moviesWithSpecifiedActors2:
    print(movie)
    print("\n")

{'_id': 'movie:9', 'title': 'Gladiator', 'actors': [{'_id': 'artist:23', 'role': 'Maximus'}, {'_id': 'artist:147', 'role': 'Commode'}, {'_id': 'artist:148', 'role': 'Lucilla'}, {'_id': 'artist:149', 'role': 'Marc Aurele'}]}




<h2>Q : Combien de films dans lesquels ont joué ni l'acteur 34, ni 98, ni 1 et afficher leur titre et la liste des actors ; sens du tri : chronologique décroissant et alphabétique croissant sur les titres des films ?</h2>

In [82]:
moviesWithoutSpecifiedActors = db.movies.find({"actors._id":{"$nin":["artist:34","artist:98","artist:1"]}},{"title" : 1, "actors": 1}).sort("year", 1).sort("actors",1)
count = 0 
for movie in moviesWithoutSpecifiedActors:
    count+=1;
    print("====="+str(count)+"=====")
    print(movie)
    print("\n")

=====1=====
{'_id': 'movie:4', 'title': 'Sacrifice', 'actors': []}


=====2=====
{'_id': 'movie:25', 'title': 'Kagemusha', 'actors': []}


=====3=====
{'_id': 'movie:40', 'title': 'FenÃªtre sur cour', 'actors': []}


=====4=====
{'_id': 'movie:64', 'title': 'Pour quelques dollars de plus', 'actors': []}


=====5=====
{'_id': 'movie:79', 'title': 'Inglourious Basterds', 'actors': []}


=====6=====
{'_id': 'movie:92', 'title': 'Nous trois ou rien', 'actors': []}


=====7=====
{'_id': 'movie:37', 'title': 'Eyes Wide Shut', 'actors': [{'_id': 'artist:65', 'role': 'Docteur William "Bill" Harford'}, {'_id': 'artist:102', 'role': 'Alice Harford'}]}


=====8=====
{'_id': 'movie:38', 'title': 'Shining', 'actors': [{'_id': 'artist:103', 'role': 'Jack Torrance'}]}


=====9=====
{'_id': 'movie:41', 'title': 'La mort aux trousses', 'actors': [{'_id': 'artist:105', 'role': 'Roger O. Thornhill'}, {'_id': 'artist:106', 'role': 'Eve Kendall'}, {'_id': 'artist:107', 'role': 'Philipp Vandamm'}]}


=====1

<h2>Q : Combien de films qui n’ont pas d’attribut actors ? Combien de films dont l'attribut actors n'est pas renseigné et les afficher ?</h2>

In [89]:
movieWithoutActors =  db.movies.find({"actors" : { "$exists": False}}).count()
print(movieWithoutActors)

movieWithEmptyActors =  db.movies.find({"actors" :{"$exists":True,"$lte":[]}},{"title" : 1, "actors": 1})
count = 0
for movie in movieWithEmptyActors:
    count+=1;
    print("====="+str(count)+"=====")
    print(movie)
    print("\n")

0
=====1=====
{'_id': 'movie:4', 'title': 'Sacrifice', 'actors': []}


=====2=====
{'_id': 'movie:25', 'title': 'Kagemusha', 'actors': []}


=====3=====
{'_id': 'movie:40', 'title': 'FenÃªtre sur cour', 'actors': []}


=====4=====
{'_id': 'movie:64', 'title': 'Pour quelques dollars de plus', 'actors': []}


=====5=====
{'_id': 'movie:79', 'title': 'Inglourious Basterds', 'actors': []}


=====6=====
{'_id': 'movie:92', 'title': 'Nous trois ou rien', 'actors': []}




<h2>Q : Donnez tous les films dont le directeur est Clint Eastwood ?</h2>

In [84]:
eastwood = db.artists.find_one({"first_name": "Clint", "last_name": "Eastwood"}) 
eastwoodMovies = db.movies.find({"director._id": eastwood['_id']}, {"title": 1, "director._id" : 1})
db.artists.fin

for movie in eastwoodMovies:
    print(movie)
    print("\n")


{'_id': 'movie:8', 'title': 'Impitoyable', 'director': {'_id': 'artist:20'}}


{'_id': 'movie:26', 'title': 'Les pleins pouvoirs', 'director': {'_id': 'artist:20'}}


{'_id': 'movie:63', 'title': 'Million Dollar Baby', 'director': {'_id': 'artist:20'}}




<h1>#####                  Expressions régulières ?                     #####</h1>

<h2>Q : Tous les films dont le titre commence par 'Re' ?</h2>

In [85]:
moviesBeginByRe = db.movies.find({"title": {"$regex" :'^Re'}}, {"actors": 0, "summary": 0})

for movie in moviesBeginByRe : 
    print(movie)



{'_id': 'movie:36', 'title': 'Reservoir dogs', 'year': 1992, 'genre': 'crime', 'country': 'USA', 'director': {'_id': 'artist:37'}}


<h2>Q : Tous les films dont le titre finit par 'la' ?</h2>

In [86]:
moviesFinishByLa = db.movies.find({"title": {"$regex" :'la$'}}, {"actors": 0, "summary": 0})

for movie in moviesFinishByLa : 
    print(movie)


{'_id': 'movie:22', 'title': 'Godzilla', 'year': 1998, 'genre': 'Action', 'country': 'USA', 'director': {'_id': 'artist:58'}}


<h2>Q : Combien de films ont été joués par l'acteur dont l'id est 147 et les afficher en se limitant au titre du film et la liste des acteurs ?</h2>

In [95]:
moviesWithSpecifiedActor = db.movies.find({"actors._id":{"$all":["artist:147"]}},{"title" : 1, "actors": 1}).sort("title",1)

for movie in moviesWithSpecifiedActor : 
    print(movie)



{'_id': 'movie:9', 'title': 'Gladiator', 'actors': [{'_id': 'artist:23', 'role': 'Maximus'}, {'_id': 'artist:147', 'role': 'Commode'}, {'_id': 'artist:148', 'role': 'Lucilla'}, {'_id': 'artist:149', 'role': 'Marc Aurele'}]}


<h2>Q : Retourner les films joués par l'acteur 147 sans les acteurs et sans le résumé ?</h2>

In [97]:
moviesWithSpecifiedActor = db.movies.find({"actors._id":{"$all":["artist:147"]}},{"summary" : 0, "actors": 0}).sort("title",1)

for movie in moviesWithSpecifiedActor : 
    print(movie)

{'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000, 'genre': 'drama', 'country': 'USA', 'director': {'_id': 'artist:4'}}
